# Week 1 - Activity 1: Simple 2-gram Language Model

In this activity, we'll implement a simple 2-gram language model with both word-level and character-level tokenization. We'll:
1. Implement tokenization
2. Collect counts
3. Calculate probabilities for new sentences
4. Add interpolation with unigram probabilities

In [1]:
from collections import defaultdict

## 1. Tokenization Functions

Let's implement both word-level and character-level tokenization with special tokens for sentence boundaries:

In [2]:
def word_tokenize(text):
    # Word tokenization with BOS and EOS tokens
    tokens = text.lower().split()
    return ['<BOS>'] + tokens + ['<EOS>']

def char_tokenize(text):
    # Character tokenization with BOS and EOS tokens
    chars = list(text.lower())
    return ['<BOS>'] + chars + ['<EOS>']

# Example usage
text = "Hello world!"
print("Word tokens:", word_tokenize(text))
print("Char tokens:", char_tokenize(text))

Word tokens: ['<BOS>', 'hello', 'world!', '<EOS>']
Char tokens: ['<BOS>', 'h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd', '!', '<EOS>']


## 2. N-gram Model with Interpolation

In [3]:
class BigramModel:
    def __init__(self, tokenize_fn, lambda_1=0.8, lambda_2=0.2):
        self.tokenize = tokenize_fn
        self.unigram_counts = defaultdict(int)
        self.bigram_counts = defaultdict(int)
        self.total_tokens = 0
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        self.vocabulary = set()
        
    def train(self, texts):
        if isinstance(texts, str):
            texts = [texts]
            
        for text in texts:
            tokens = self.tokenize(text)
            
            # Build vocabulary and count unigrams
            for token in tokens:
                self.vocabulary.add(token)
                self.unigram_counts[token] += 1
                self.total_tokens += 1
            
            # Count bigrams
            for i in range(len(tokens)-1):
                bigram = (tokens[i], tokens[i+1])
                self.bigram_counts[bigram] += 1
    
    def get_unigram_prob(self, token):
        # Handle unseen tokens with a small probability
        if token not in self.vocabulary:
            return 1.0 / (self.total_tokens + len(self.vocabulary))
        return self.unigram_counts[token] / self.total_tokens
    
    def get_bigram_prob(self, token1, token2):
        bigram = (token1, token2)
        if token1 not in self.vocabulary:
            return 0.0
        denominator = self.unigram_counts[token1]
        if denominator == 0:
            return 0.0
        return self.bigram_counts[bigram] / denominator
    
    def get_interpolated_prob(self, token1, token2):
        bigram_prob = self.get_bigram_prob(token1, token2)
        unigram_prob = self.get_unigram_prob(token2)
        return self.lambda_1 * bigram_prob + self.lambda_2 * unigram_prob
    
    def get_sequence_prob(self, text, use_interpolation=True):
        tokens = self.tokenize(text)
        if len(tokens) < 2:
            return 0.0
        
        prob = 1.0
        for i in range(len(tokens)-1):
            if use_interpolation:
                prob *= self.get_interpolated_prob(tokens[i], tokens[i+1])
            else:
                prob *= self.get_bigram_prob(tokens[i], tokens[i+1])
        return prob
    
    def analyze_sequence(self, text, use_interpolation=True):
        tokens = self.tokenize(text)
        print(f"\nAnalyzing sequence: {text}")
        print("Token sequence:", tokens)
        print("\nProbability breakdown:")
        
        total_prob = 1.0
        for i in range(len(tokens)-1):
            token1, token2 = tokens[i], tokens[i+1]
            
            if use_interpolation:
                bigram_prob = self.get_bigram_prob(token1, token2)
                unigram_prob = self.get_unigram_prob(token2)
                interpolated_prob = self.get_interpolated_prob(token1, token2)
                
                print(f"\nTransition {token1} → {token2}:")
                print(f"  Bigram P({token2}|{token1}) = {bigram_prob:.4f}")
                print(f"  Unigram P({token2}) = {unigram_prob:.4f}")
                print(f"  Interpolated = {self.lambda_1:.2f} * {bigram_prob:.4f} + {self.lambda_2:.2f} * {unigram_prob:.4f} = {interpolated_prob:.4f}")
                
                total_prob *= interpolated_prob
            else:
                prob = self.get_bigram_prob(token1, token2)
                print(f"P({token2}|{token1}) = {prob:.4f}")
                total_prob *= prob
                
        print(f"\nTotal sequence probability: {total_prob:.6f}")
        return total_prob

## 3. Training and Testing

Let's train both word-level and character-level models on some example text:

In [4]:
# Training data
training_texts = [
    "the cat sat on the mat",
    "the dog ran in the park",
    "a cat and a dog played"
]

# Create and train word-level model
word_model = BigramModel(word_tokenize)
word_model.train(training_texts)

# Create and train character-level model
char_model = BigramModel(char_tokenize, lambda_1=0.7, lambda_2=0.3)
char_model.train(training_texts)

## 4. Testing Different Sequences

Let's test both models on some sentences and compare their probabilities:

In [5]:
test_sequences = [
    "the cat sat",    # Seen sequence
    "xyz",            # Completely unseen
    "the xyz cat"     # Partially seen
]

print("Comparing interpolated probabilities for different sequences:")
for sequence in test_sequences:
    print(f"\n{'='*50}")
    print(f"Sequence: {sequence}")
    
    print("\nWord-level analysis:")
    word_model.analyze_sequence(sequence)
    
    print("\nCharacter-level analysis:")
    char_model.analyze_sequence(sequence)

Comparing interpolated probabilities for different sequences:

Sequence: the cat sat

Word-level analysis:

Analyzing sequence: the cat sat
Token sequence: ['<BOS>', 'the', 'cat', 'sat', '<EOS>']

Probability breakdown:

Transition <BOS> → the:
  Bigram P(the|<BOS>) = 0.6667
  Unigram P(the) = 0.1667
  Interpolated = 0.80 * 0.6667 + 0.20 * 0.1667 = 0.5667

Transition the → cat:
  Bigram P(cat|the) = 0.2500
  Unigram P(cat) = 0.0833
  Interpolated = 0.80 * 0.2500 + 0.20 * 0.0833 = 0.2167

Transition cat → sat:
  Bigram P(sat|cat) = 0.5000
  Unigram P(sat) = 0.0417
  Interpolated = 0.80 * 0.5000 + 0.20 * 0.0417 = 0.4083

Transition sat → <EOS>:
  Bigram P(<EOS>|sat) = 0.0000
  Unigram P(<EOS>) = 0.1250
  Interpolated = 0.80 * 0.0000 + 0.20 * 0.1250 = 0.0250

Total sequence probability: 0.001253

Character-level analysis:

Analyzing sequence: the cat sat
Token sequence: ['<BOS>', 't', 'h', 'e', ' ', 'c', 'a', 't', ' ', 's', 'a', 't', '<EOS>']

Probability breakdown:

Transition <BOS> → t:

## Discussion Points

1. Compare the probabilities from word-level and character-level models:
   - Why are they so different in scale?
   - How does interpolation help with unseen sequences?

2. Which model (word or character) is better at handling:
   - Common phrases?
   - Novel words?
   - Partially seen sequences?

3. What are the effects of different interpolation weights (λ₁ and λ₂)?
   - Higher λ₁: More weight on context (bigrams)
   - Higher λ₂: More weight on individual tokens (unigrams)

4. How could we further improve this model?
   - Using higher-order n-grams
   - Different smoothing techniques
   - Subword tokenization
   - Neural language models